In [4]:
!nvidia-smi

# If this doesn't work, there's no GPU available or detected

Sat Feb 18 21:56:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8    15W /  60W |      8MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install audiolm-pytorch
#!pip install --upgrade --force-reinstall audiolm-pytorch

  Using cached audiolm_pytorch-0.14.3-py3-none-any.whl (29 kB)
  Using cached accelerate-0.16.0-py3-none-any.whl (199 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached fairseq-0.12.2-cp310-cp310-linux_x86_64.whl
  Using cached Mega_pytorch-0.0.15-py3-none-any.whl (6.4 kB)
  Using cached beartype-0.12.0-py3-none-any.whl (754 kB)
  Using cached ema_pytorch-0.2.1-py3-none-any.whl (4.4 kB)
  Using cached lion_pytorch-0.0.6-py3-none-any.whl (4.2 kB)
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached torchaudio-0.13.1-cp310-cp310-manylinux1_x86_64.whl (4.2 MB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached sentencepiece-0.1.97-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached scikit_learn-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached local_attention-1.6.0-py3-none-any.whl (7

## Setup

### Imports & paths

In [2]:
# imports
import math
import wave
import struct
import os
import urllib.request
import tarfile
from audiolm_pytorch import SoundStream, SoundStreamTrainer, HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer, HubertWithKmeans, CoarseTransformer, CoarseTransformerWrapper, CoarseTransformerTrainer, FineTransformer, FineTransformerWrapper, FineTransformerTrainer, AudioLM
from torch import nn
import torch
import torchaudio


# define all dataset paths, checkpoints, etc
dataset_folder = "../../datasets/fma_medium/"
soundstream_ckpt = "runs/soundstream.20200_320.pt" # this can change depending on number of steps
hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = f'hubert/hubert_base_ls960_L9_km500.bin' # listed in row "HuBERT Base (~95M params)", column Quantizer

## Training

Now that we have a dataset, we can train AudioLM.

**Note**: do NOT type "y" to overwrite previous experiments/ checkpoints when running through the cells here unless you're ready to the entire results folder! Otherwise you will end up erasing things (e.g. you train SoundStream first, and if you choose "overwrite" then you lose the SoundStream checkpoint when you then train SemanticTransformer).

### SoundStream

In [3]:
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
    target_sample_hz = 24000,
    strides = (2, 4, 5, 8)
)

soundstream.load_from_trainer_saved_obj(f"./{soundstream_ckpt}")  #Load pretrained

trainer = SoundStreamTrainer(
    soundstream,
    folder = dataset_folder,
    batch_size = 4,
    grad_accum_every = 8,         # effective batch size of 32
    data_max_length = 320 * 32,
    save_results_every = 100,
    save_model_every = 100,
    num_train_steps = 25005,
    random_split_seed=1112
).cuda()

trainer.train()

training with dataset of 23729 samples and validating with randomly splitted 1249 samples
0: saving to results
0: saving model to results
100: saving to results
100: saving model to results
200: saving to results
200: saving model to results
300: saving to results
300: saving model to results
400: saving to results
400: saving model to results
500: saving to results
500: saving model to results
600: saving to results
600: saving model to results
700: saving to results
700: saving model to results
800: saving to results
800: saving model to results
900: saving to results
900: saving model to results
1000: saving to results
1000: saving model to results
1100: saving to results
1100: saving model to results
1200: saving to results
1200: saving model to results
1300: saving to results
1300: saving model to results
1400: saving to results
1400: saving model to results
1500: saving to results
1500: saving model to results
1600: saving to results
1600: saving model to results
1700: saving to 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_730190/341436423.py:22 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_730190/341436423.py'                        │
│                                                                                                  │
│ /home/lagodish/Dev/audiolm-pytorch/audiolm_pytorch/trainer.py:454 in train                       │
│                                                                                                  │
│    451 │   def train(self, log_fn = noop):                                                       │
│    452 │   │                                                                                     │
│    453 │   │   while self.steps < self.num_train_steps:                                          │
│ ❱  454 │   │   │   logs = self.train_step()                                                      │
│    455 │   │   │   log_fn(logs)                                                                  │
│    456 │   │                                                                                     │
│    457 │   │   self.print('training complete')                                                   │
│                                                                                                  │
│ /home/lagodish/Dev/audiolm-pytorch/audiolm_pytorch/trainer.py:368 in train_step                  │
│                                                                                                  │
│    365 │   │   │                                                                                 │
│    366 │   │   │   for name, discr_loss in discr_losses:                                         │
│    367 │   │   │   │   self.accelerator.backward(discr_loss / self.grad_accum_every, retain_gra  │
│ ❱  368 │   │   │   │   accum_log(logs, {name: discr_loss.item() / self.grad_accum_every})        │
│    369 │   │                                                                                     │
│    370 │   │   if exists(self.discr_max_grad_norm):                                              │
│    371 │   │   │   self.accelerator.clip_grad_norm_(self.soundstream.stft_discriminator.paramet  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

### SoundStream Check

In [40]:
from audiolm_pytorch.data import SoundDataset, get_dataloader
from audiolm_pytorch.trainer import cycle
from pathlib import Path

ds = SoundDataset(
            folder = dataset_folder,
            max_length = 24000*15,  #15 Sec
            target_sample_hz = soundstream.target_sample_hz,
            seq_len_multiple_of = soundstream.seq_len_multiple_of
)

dl = get_dataloader(ds, batch_size = 1, num_workers = 0, shuffle = True)

dl_iter = cycle(dl)

wave, = next(dl_iter)
wave = wave.to("cuda")


results_folder = Path('./results_test')
results_folder.mkdir(exist_ok=True)
filename = str(results_folder / f'orig.wav')
torchaudio.save(filename, wave.cpu(), soundstream.target_sample_hz)

In [41]:
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 12,
    target_sample_hz = 24000,
    strides = (3, 4, 5, 8)
)

for i in range(9300,9400,100):
    soundstream_path = f"results/soundstream.{i}.pt"
    soundstream.load_from_trainer_saved_obj(f"./{soundstream_path}")  #Load ckpt for test

    soundstream.eval()
    soundstream = soundstream.to("cuda")

    with torch.no_grad():
        recons = soundstream(wave, return_recons_only = True)

    for ind, recon in enumerate(recons.unbind(dim = 0)):
        filename = str(results_folder / f'sample_{i}.wav')
        torchaudio.save(filename, recon.cpu().detach(), soundstream.target_sample_hz)

### SemanticTransformer

In [18]:
# hubert checkpoints can be downloaded at
# https://github.com/facebookresearch/fairseq/tree/main/examples/hubert
if not os.path.isdir("hubert"):
  os.makedirs("hubert")
if not os.path.isfile(hubert_ckpt):
  hubert_ckpt_download = f"https://dl.fbaipublicfiles.com/{hubert_ckpt}"
  urllib.request.urlretrieve(hubert_ckpt_download, f"./{hubert_ckpt}")
if not os.path.isfile(hubert_quantizer):
  hubert_quantizer_download = f"https://dl.fbaipublicfiles.com/{hubert_quantizer}"
  urllib.request.urlretrieve(hubert_quantizer_download, f"./{hubert_quantizer}")

wav2vec = HubertWithKmeans(
    checkpoint_path = f'./{hubert_ckpt}',
    kmeans_path = f'./{hubert_quantizer}'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6
).cuda()


trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    folder = dataset_folder,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 1
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 0.24.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


training with dataset of 2 samples and validating with randomly splitted 1 samples
do you want to clear previous experiment checkpoints and results? (y/n) n
0: loss: 6.648584365844727
0: valid loss 5.763116359710693
0: saving model to results
training complete


### CoarseTransformer

In [19]:
wav2vec = HubertWithKmeans(
    checkpoint_path = f'./{hubert_ckpt}',
    kmeans_path = f'./{hubert_quantizer}'
)

soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)

soundstream.load(f"./{soundstream_ckpt}")

coarse_transformer = CoarseTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    codebook_size = 1024,
    num_coarse_quantizers = 3,
    dim = 512,
    depth = 6
)

trainer = CoarseTransformerTrainer(
    transformer = coarse_transformer,
    soundstream = soundstream,
    wav2vec = wav2vec,
    folder = dataset_folder,
    batch_size = 1,
    data_max_length = 320 * 32,
    save_results_every = 2,
    save_model_every = 4,
    num_train_steps = 9
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason

trainer.train()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator MiniBatchKMeans from version 0.24.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


training with dataset of 2 samples and validating with randomly splitted 1 samples
do you want to clear previous experiment checkpoints and results? (y/n) n
0: loss: 63.983970642089844
0: valid loss 63.398582458496094
0: saving model to results
1: loss: 65.85967254638672
2: loss: 62.4722900390625
2: valid loss 50.01605987548828
3: loss: 11.735434532165527
4: loss: 3.976104497909546
4: valid loss 46.094608306884766
4: saving model to results
5: loss: 58.27140426635742
6: loss: 41.68347930908203
6: valid loss 45.54595184326172
7: loss: 2.2387890815734863
8: loss: 0.4718627631664276
8: valid loss 39.10848617553711
8: saving model to results
training complete


### FineTransformer

In [20]:
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)

soundstream.load(f"./{soundstream_ckpt}")

fine_transformer = FineTransformer(
    num_coarse_quantizers = 3,
    num_fine_quantizers = 5,
    codebook_size = 1024,
    dim = 512,
    depth = 6
)

trainer = FineTransformerTrainer(
    transformer = fine_transformer,
    soundstream = soundstream,
    folder = dataset_folder,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 9
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason

trainer.train()

training with dataset of 2 samples and validating with randomly splitted 1 samples
do you want to clear previous experiment checkpoints and results? (y/n) n
0: loss: 70.90608215332031
0: valid loss 65.99951171875
0: saving model to results
1: loss: 43.6014289855957
2: loss: 8.300681114196777
3: loss: 61.23375701904297
4: loss: 63.34052276611328
5: loss: 2.010118246078491
6: loss: 56.52588653564453
7: loss: 0.5423888564109802
8: loss: 0.005095238331705332
training complete


## Inference

In [21]:
# Everything together
audiolm = AudioLM(
    wav2vec = wav2vec,
    soundstream = soundstream,
    semantic_transformer = semantic_transformer,
    coarse_transformer = coarse_transformer,
    fine_transformer = fine_transformer
)

generated_wav = audiolm(batch_size = 1)

generating fine: 100%|██████████| 512/512 [02:56<00:00,  2.91it/s]


In [22]:
output_path = "out.wav"
sample_rate = 44100
torchaudio.save(output_path, generated_wav.cpu(), sample_rate)